In [1]:
import os
import pandas as pd
from pathlib import Path

Data taxis verdes

Esta funcion sirve para realizar la carga de datos

In [2]:
def load_parquet_files(folder_path, Año):
    parquet_files = Path(folder_path).glob('*.parquet')#Obtenemos una lista de rutas a archivos .parquet en la carpeta especificada
    parquet_files_Año = [file for file in parquet_files if f'green_tripdata_{Año}' in file.name]#filtramos el nombre del archivo con el que vamos trabajar

    data_frames_Año = []#iniciamos un lista para almacenar los DataFrames cargados desde los archivos .parquet
    for file in parquet_files_Año:# Iteramos a través de los archivos .parquet correspondientes al año y cargar cada uno en un DataFrame
        df = pd.read_parquet(file)# Cargamos el archivo parquet
        data_frames_Año.append(df)# Agregamos el DataFrame cargado a la lista

    return data_frames_Año # Devolvemos la lista de DataFrames correspondientes al año


Función para transformar datos

In [4]:
def transform_data(data_frames, Año):
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['lpep_pickup_datetime'] = pd.to_datetime(combined_df['lpep_pickup_datetime'])
    combined_df['pickup_date'] = combined_df['lpep_pickup_datetime'].dt.date

    df_taxi = combined_df.copy()
    df_taxi['pickup_date'] = pd.to_datetime(df_taxi['pickup_date'])

    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxi.columns if col not in ['VendorID', 'pickup_date']]
    df_taxi = df_taxi[column_order]

    df_taxi = df_taxi[df_taxi['pickup_date'].dt.year == Año]

    columns_to_remove = ['lpep_pickup_datetime','lpep_dropoff_datetime','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','payment_type','trip_type','extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxi = df_taxi.drop(columns=columns_to_remove)

    # Agregar una columna que indica el día de la semana (0: Lunes, 6: Domingo)
    df_taxi['weekday'] = df_taxi['pickup_date'].dt.weekday

    # Mapear los números de día de la semana a nombres de días
    day_names = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    df_taxi['weekday'] = df_taxi['weekday'].apply(lambda x: day_names[x])

    # Agrupar por fecha y calcular el total_amount por día
    df_taxi_grouped = df_taxi.groupby(['pickup_date', 'weekday'])['total_amount'].sum().reset_index()

    # Agregar una columna que indica la cantidad de veces que se repite una fecha
    df_taxi_grouped['row_count'] = df_taxi['pickup_date'].value_counts()[df_taxi_grouped['pickup_date']].values

    return df_taxi_grouped

Realizamos los llamados a las funciones 

In [8]:

folder_path = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo'
Año = 2023
data_frames_Año = load_parquet_files(folder_path, Año)
df_taxi_grouped = transform_data(data_frames_Año, Año)
df_taxi_grouped.head(10)
#Guardamos el DataFrame con el nombre correspondiente al año elegido
df_taxi_grouped.to_csv(f'taxiV_{Año}.csv') 


Data Taxis amarillos

Se realiza el mismo procedimiento que en la funcion anterior, con la diferencia que al ser archivos mas pesados se deben crear DataFrame por día

Función para la carga de datos

In [24]:
def load_parquet_filesY(folder_path, AñoY):
    parquet_files = Path(folder_path_Y).glob('*.parquet')
    parquet_files_Año = [file for file in parquet_files if f'yellow_tripdata_{AñoY}-0{mes}' in file.name]

    data_frames_AñoY = []
    for file in parquet_files_Año:
        df = pd.read_parquet(file)
        data_frames_AñoY.append(df)

    return data_frames_AñoY


Funcion de Transformación de datos

In [42]:
def transform_data(data_frames, AñoY):
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['tpep_pickup_datetime'] = pd.to_datetime(combined_df['tpep_pickup_datetime'])
    combined_df['pickup_date'] = combined_df['tpep_pickup_datetime'].dt.date

    df_taxiY = combined_df.copy()
    df_taxiY['pickup_date'] = pd.to_datetime(df_taxiY['pickup_date'])

    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxiY.columns if col not in ['VendorID', 'pickup_date']]
    df_taxiY = df_taxiY[column_order]

    df_taxiY = df_taxiY[df_taxiY['pickup_date'].dt.year == AñoY]

    columns_to_remove = ['VendorID', 'Airport_fee','tpep_pickup_datetime','tpep_dropoff_datetime','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','payment_type','extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxiY = df_taxiY.drop(columns=columns_to_remove)

    # Agregar una columna que indica el día de la semana (0: Lunes, 6: Domingo)
    df_taxiY['weekday'] = df_taxiY['pickup_date'].dt.weekday

    # Mapear los números de día de la semana a nombres de días
    day_names = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    df_taxiY['weekday'] = df_taxiY['weekday'].apply(lambda x: day_names[x])

    # Agrupar por fecha y calcular el total_amount por día
    df_taxi_grouped = df_taxiY.groupby(['pickup_date', 'weekday'])['total_amount'].sum().reset_index()

    # Agregar una columna que indica la cantidad de veces que se repite una fecha
    df_taxi_grouped['row_count'] = df_taxiY.groupby('pickup_date').size().reset_index(name='row_count')['row_count']

    return df_taxi_grouped


Llamado a las funciones y guardado de los DataFrame

In [46]:
folder_path_Y = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo\\Amarillo'
AñoY = 2023
mes=5
data_frames = load_parquet_filesY(folder_path_Y, AñoY)
df_taxiY = transform_data(data_frames, AñoY)
df_taxiY.head(10)
df_taxiY.to_csv(f'taxiY_{AñoY}_{mes}.csv')

Combinacion de DataFrame

Función de carga de datos

In [83]:
def load_csv_files(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and f'taxiV_' in file]
    
    data_frames = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    
    return data_frames

Función de transformación de DF

In [86]:
def combine_and_transform_data(data_frames):
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    columns_to_drop = ["Unnamed: 0"]
    combined_df = combined_df.drop(columns=columns_to_drop)
    
    return combined_df

In [87]:
folder_path = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo'

data_frames = load_csv_files(folder_path)
combined_data = combine_and_transform_data(data_frames)
combined_data.to_csv('taxiG.csv')

In [88]:
dfv=pd.read_csv('taxiG.csv')

In [89]:
dfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    884 non-null    int64  
 1   pickup_date   884 non-null    object 
 2   weekday       884 non-null    object 
 3   total_amount  884 non-null    float64
 4   row_count     884 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 34.7+ KB


Estaciones de carga

In [ ]:
import numpy as np
import datetime as dt 

Creo el DataFrame

In [ ]:
dfElectric=pd.read_csv('Electric and Alternative Fuel Charging Stations.csv')

Chequeo te lo que tiene dentro

In [ ]:
dfElectric.info()

In [ ]:
dfElectric.head()

Transformo el DataFrame

In [ ]:
#Filtro por ubicación
estaciones_ny = dfElectric[dfElectric['State'] == 'NY']
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['Street Address','Intersection Directions','ZIP','Plus4','Station Phone','Status Code','Groups With Access Code',
'Access Days Time','Cards Accepted','Date Last Confirmed','Updated At','Owner Type Code','Federal Agency ID',
'Open Date','Country','Access Code','Facility Type','CNG On-Site Renewable Source','CNG Total Compression Capacity','CNG Storage Capacity','EV Pricing',
'LPG Nozzle Types','CNG Fill Type Code','CNG PSI','EV On-Site Renewable Source','Restricted Access','Expected Date','BD Blends','NG Fill Type Code','NG PSI',
'EV Other Info','EV Network Web','Hydrogen Status Link','LPG Primary', 'E85 Blender Pump', 'Intersection Directions (French)','Access Days Time (French)','BD Blends (French)',
'Hydrogen Is Retail','Federal Agency Code','LNG On-Site Renewable Source','E85 Other Ethanol Blends','EV Pricing (French)','Hydrogen Pressures','Hydrogen Standards','Federal Agency Name'
]

# Elimino las columnas especificadas
estaciones_ny = estaciones_ny.drop(columns=columnas_a_eliminar)
# Reorganizo las columnas
column_order = ['ID'] + [col for col in estaciones_ny.columns if col != 'ID']
estaciones_ny=estaciones_ny[column_order]

Guardo el DataFrame creado

In [ ]:
estaciones_ny.to_csv('Station_NY.csv',index=False)

Zona de Taxis

Creo el DataFrame

In [ ]:
dftaxi_zone=pd.read_csv('Dataset_extraidos\\taxi_zones.csv')

Reviso la información que contiene

In [ ]:
dftaxi_zone.info()

In [ ]:
dftaxi_zone.head()

Trasformación

In [ ]:
columns_to_drop = ['Unnamed: 0', 'OBJECTID']
dftaxi_zone = dftaxi_zone.drop(columns=columns_to_drop)

In [ ]:
column_order = ['LocationID'] + [col for col in dftaxi_zone.columns if col != 'LocationID']
dftaxi_zone=dftaxi_zone[column_order]

In [ ]:
dftaxi_zone = dftaxi_zone.rename(columns={'y': 'longitude', 'x': 'latitude'})

Guardado

In [ ]:
dftaxi_zone.to_csv('Taxi zone.csv',index=False)